Import packages:

In [1]:
import pickle
import itertools
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import patsy
from sklearn import preprocessing, linear_model, model_selection, ensemble, metrics
from ggplot import *
from tqdm import tnrange, tqdm_notebook

/home/hobbs/.local/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/hobbs/.local/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Now we need to import our data. I'm going to import it once from the csv, and create a pickle below. However, when I re-run this notebook I want to be able to skip steps, so I'll just load the pickle. That is much faster. The argument `quick=False` will force the program to re-create the pickle (for example if we change the csv data or add new variables to the load_data function).

In [2]:
def load_data(quick=True):
    
    def make_pickle():
        # Read csv
        df = pd.read_csv('nodes_final_data.csv')
        
        # Set the index
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index(['node', 'time'])

        # Create/ format some variables
        df['week'] = [value[1].isocalendar()[1] for value in df.index.values]

        # Save as a pickle
        df.to_pickle('nodes_final_data.p')
        
        return df
        
    if quick == True:
        try:
            print("Trying to open saved data.")
            with open('nodes_final_data.p', 'rb') as f:
                return pickle.load(f)
        
        except FileNotFoundError:
            print("No existing pickle found... picklemaking!")
            return make_pickle()
    else:
        print("Pickling a fresh new pickle.")
        return make_pickle()

def load_subset(n, df):
    print("Loading a subset with ", n, " nodes.")
    np.random.seed(seed=1)
    node_ids = df.index.get_level_values('node').unique()
    selected_nodes = list(np.random.choice(node_ids, size = n))
    return df.loc[selected_nodes]

In [3]:
df = load_subset(100, load_data())
df.head()

Trying to open saved data.
Loading a subset with  100  nodes.


year  month  date  hr  opr_hr  day  \
node         time                                                      
ALAMT4G_7_B1 2017-05-02 00:00:00  2017      5     1   0      18    2   
             2017-05-02 01:00:00  2017      5     1   1      19    2   
             2017-05-02 02:00:00  2017      5     1   2      20    2   
             2017-05-02 03:00:00  2017      5     1   3      21    2   
             2017-05-02 04:00:00  2017      5     1   4      22    2   

                                  dollar_mw      other_MW     solar_MW  \
node         time                                                        
ALAMT4G_7_B1 2017-05-02 00:00:00  30.549540  13091.870117  6837.720215   
             2017-05-02 01:00:00  47.267689  17182.660156  2931.449951   
             2017-05-02 02:00:00  71.159882  19997.630859   264.540009   
             2017-05-02 03:00:00  66.884888  19904.410156     0.000000   
             2017-05-02 04:00:00  48.442291  17607.289062     0.000000   

                                      wind_MW  load_MW  fuel_price  \
node         time                                                    
ALAMT4G_7_B1 2017-05-02 00:00:00  1992.119995  28981.0        2.83   
             2017-05-02 01:00:00  2254.040039  29669.0        2.83   
             2017-05-02 02:00:00  2461.199951  29458.0        2.83   
             2017-05-02 03:00:00  2499.290039  30325.0        2.83   
             2017-05-02 04:00:00  2473.360107  29429.0        2.83   

                                   net_exp_MW        temp       irrad  \
node         time                                                       
ALAMT4G_7_B1 2017-05-02 00:00:00 -8073.870117  292.096466  294.044172   
             2017-05-02 01:00:00 -8380.599609  292.121277  294.037069   
             2017-05-02 02:00:00 -8663.000000  292.146118  294.029965   
             2017-05-02 03:00:00 -8644.000000  292.170959  294.022861   
             2017-05-02 04:00:00 -8591.799805  292.195801  294.015758   

                                    wind_u    wind_v   latitude   longitude  \
node         time                                                             
ALAMT4G_7_B1 2017-05-02 00:00:00  2.531747  1.636286  33.765897 -118.103607   
             2017-05-02 01:00:00  2.531789  1.636183  33.765897 -118.103607   
             2017-05-02 02:00:00  2.531830  1.636081  33.765897 -118.103607   
             2017-05-02 03:00:00  2.531872  1.635978  33.765897 -118.103607   
             2017-05-02 04:00:00  2.531913  1.635875  33.765897 -118.103607   

                                  week  
node         time                       
ALAMT4G_7_B1 2017-05-02 00:00:00    18  
             2017-05-02 01:00:00    18  
             2017-05-02 02:00:00    18  
             2017-05-02 03:00:00    18  
             2017-05-02 04:00:00    18

# Feature Making

Now we're going to expand our data into more features. Patsy is good at this, but like the above, it is time consuming to do it repeatedly. Thus, we'll save the results and only recreate them if something changes. The only downside of this is potential proliferation of pickle-matrices - gotta delete these occassionally!

In [4]:
def quick_patsy(arg, input_data, quick=True):
     #File will save with the patsy description and number of observations
    filename = arg + str(input_data.shape[1]) + '.p'
    if quick:
        try:
            with open(filename, 'rb') as f:
                y, X = pickle.load(f)
        except FileNotFoundError:
            with open(filename, 'wb') as f:
                y, X = tuple(np.array(matrix) for matrix in patsy.dmatrices(arg, data = input_data))
                pickle.dump((y, X), f)
    else:
        with open(filename, 'wb') as f:
                y, X = tuple(np.array(matrix) for matrix in patsy.dmatrices(arg, data = input_data))
                pickle.dump((y, X), f)
                
    y = np.array(y)
    X = np.array(X)
    return (y, X)

We also want to create time and spatial lags. To do that, we can use `shift()` in combination with `groupby()`.

In [5]:
normalize = lambda x: (x - np.mean(x))/ np.std(x)

def lag_var(df, var, n_periods):
    return df[var].groupby(level='node').shift(n_periods)

df['temp_last_hr'] = lag_var(df, 'temp', 1)
df['price_last_hr'] = lag_var(df, 'dollar_mw', 1)
df['price_yesterday'] = lag_var(df, 'dollar_mw', 24)
df['price_last_week'] = lag_var(df, 'dollar_mw', 24 * 7)
df['nodenorm_temp'] = df['temp'].groupby(level = 'node').apply(normalize)
df['node'] = [value[0] for value in df.index.values]
print("Normalizing price")

lagnames = ''

for i in list(range(1, 24 * 7)):
    name = 'lag' + str(i)
    lagnames += name + ' + '
    df[name] = lag_var(df, 'dollar_mw', i)

# temperature bins
#bins = [np.min(df['nodenorm_temp']), -2, -1, 1, 2, np.max(df['nodenorm_temp'])]
#group_names = ['Very Low', 'Low', 'Normal', 'High', 'Very High']
#df['temp_bin'] = pd.cut(df['nodenorm_temp'], bins, labels=group_names)

# drop NA values, since beginning and ends now lack variables
df = df.dropna()

# normalize features
#to_normalize = ['other_MW', 'solar_MW', 'wind_MW', 'latitude', 'longitude', 'temp']
#df[to_normalize] = df[to_nbormalize].apply(normalize)

df.head()

Normalizing price


year  month  date  hr  opr_hr  day  \
node         time                                                      
ALAMT4G_7_B1 2017-05-09 00:00:00  2017      5     8   0      18    2   
             2017-05-09 01:00:00  2017      5     8   1      19    2   
             2017-05-09 02:00:00  2017      5     8   2      20    2   
             2017-05-09 03:00:00  2017      5     8   3      21    2   
             2017-05-09 04:00:00  2017      5     8   4      22    2   

                                  dollar_mw      other_MW     solar_MW  \
node         time                                                        
ALAMT4G_7_B1 2017-05-09 00:00:00  32.478378  14447.700195  5280.080078   
             2017-05-09 01:00:00  45.811298  17177.009766  2384.699951   
             2017-05-09 02:00:00  66.499413  20402.220703   220.619995   
             2017-05-09 03:00:00  67.674500  20218.810547     0.000000   
             2017-05-09 04:00:00  48.831501  17610.689453     0.000000   

                                      wind_MW    ...         lag158  \
node         time                                ...                  
ALAMT4G_7_B1 2017-05-09 00:00:00   722.780029    ...      17.720961   
             2017-05-09 01:00:00  1037.520020    ...      27.224449   
             2017-05-09 02:00:00  1573.640015    ...      39.247700   
             2017-05-09 03:00:00  1875.969971    ...      51.065639   
             2017-05-09 04:00:00  2143.800049    ...      39.964081   

                                     lag159     lag160     lag161     lag162  \
node         time                                                              
ALAMT4G_7_B1 2017-05-09 00:00:00  12.146050  24.534060  27.252560  33.477322   
             2017-05-09 01:00:00  17.720961  12.146050  24.534060  27.252560   
             2017-05-09 02:00:00  27.224449  17.720961  12.146050  24.534060   
             2017-05-09 03:00:00  39.247700  27.224449  17.720961  12.146050   
             2017-05-09 04:00:00  51.065639  39.247700  27.224449  17.720961   

                                     lag163     lag164     lag165     lag166  \
node         time                                                              
ALAMT4G_7_B1 2017-05-09 00:00:00  36.006779  48.442291  66.884888  71.159882   
             2017-05-09 01:00:00  33.477322  36.006779  48.442291  66.884888   
             2017-05-09 02:00:00  27.252560  33.477322  36.006779  48.442291   
             2017-05-09 03:00:00  24.534060  27.252560  33.477322  36.006779   
             2017-05-09 04:00:00  12.146050  24.534060  27.252560  33.477322   

                                     lag167  
node         time                            
ALAMT4G_7_B1 2017-05-09 00:00:00  47.267689  
             2017-05-09 01:00:00  71.159882  
             2017-05-09 02:00:00  66.884888  
             2017-05-09 03:00:00  48.442291  
             2017-05-09 04:00:00  36.006779  

[5 rows x 193 columns]

Now we need to hold out a true test set:

In [96]:
df['day']  = [date.isocalendar()[1] for date in df.index.get_level_values('time')]

np.random.seed(seed=1)
# select 10 random test days
test_days = list(np.random.choice(df['week'].unique(), size = 2))

# create test set
df_test = df[df['week'].isin(test_weeks)]

#create training set
df = df[~df['week'].isin(test_weeks)]

# Model Runner

This function is designed to run models and record the same results for all of them. Changing the code here will change how all models are run.

In [131]:
def rmse(y, y_hat):
    return np.sqrt(np.mean(np.power(np.subtract(y, y_hat), 2)))

def wmae(little_df, df):
    # name is wrong as an artifact of how the thing was produced
    little_df = little_df.rename(columns={'dollar_mw': 'error'})
    # merge with prices
    little_df = pd.merge(little_df, df, left_index=True, right_index=True)
    
    # Get absolute value of the error
    little_df['abs_error'] = np.absolute(little_df['error'])
    # Get week index for grouping
    little_df['week']  = [date.isocalendar()[1] for date in little_df.index.get_level_values('time')]
    return little_df.groupby('week').mean()['abs_error']/little_df.groupby('week').mean()['dollar_mw']

def evaluate(train_index, test_index, model, X, y):
    # Split into train and test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Save indices from y
    index_values = y_test.index
    
    fitted = model.fit(X_train, np.ravel(y_train))
    # Calculate y_hats and map to indices
    y_hat = pd.DataFrame(data = fitted.predict(X_test), index = index_values)
    errors = np.subtract(y_test, y_hat)
    y_hat_all = fitted.predict(X)
    return [rmse(y_test, y_hat),  y_hat, errors]

def run_models(models, feature_sets, df, folds=8, parallel=True):
    '''
    Takes a list of models and features, and runs each model with each set of features
    Features should be patsy-formatted strings. 
    
    Models should be a list of sci-kit learn models and the second
    the number of jobs that used for cross-validation.
    
    Data is 'df' (a pandas dataframe), and 'folds' is the number of folds that should be used
    for cross-validation.
    '''
    #iterate through all the models
    results = []
    error_list = []
    kf = model_selection.KFold(n_splits=folds)
    for features in tqdm_notebook(feature_sets, desc = 'Feature Set'):
        y, X = patsy.dmatrices(features, data=df, return_type = 'dataframe')
        # Normalize X
        X = preprocessing.scale(X)
        for model in tqdm_notebook(models, desc = "Models"):
            if parallel:
                result = Parallel(n_jobs=folds)(delayed(evaluate)(train_index, test_index, model, X, y) for train_index, test_index in kf.split(X))
            else:
                result = [evaluate(train_index, test_index, model, X, y) for train_index, test_index in kf.split(X)]
            scores = [res[0] for res in result]
            errors = [res[2] for res in result]
            y_hat =  [res[1] for res in result]
            results.append({'model': model, 
                            'features': features, 
                            'score': np.mean(scores)})
            error_list.append(errors)
        
    return {'results': pd.DataFrame(results), 'errors': error_list}

# Linear Models

Since linear models have to be linear, it makes sense to run them with different (larger) sets of features. For example, adding squared and interaction terms makes more sense. A random forest could achieve this kind of linearity without being given the transformed variables, so there's no need to provide it.

In [100]:
ols = [linear_model.LinearRegression(fit_intercept=True, n_jobs = 3)]

#generate a huge list of different elastic nets
elastic_nets = [linear_model.ElasticNet(alpha= a, l1_ratio = l, warm_start = True)
                for a, l in list(itertools.product(np.linspace(0.5, 1, num = 3), np.linspace(0,1, num = 3)))]

feature_ideas = ['dollar_mw ~ price_last_hr + price_yesterday + price_last_week',
                 'dollar_mw ~ C(node) + price_last_hr + price_yesterday + price_last_week + week + np.power(week,2) + solar_MW + wind_MW + latitude + longitude',
                 'dollar_mw ~ C(node) + price_last_hr + price_yesterday + price_last_week + week + np.power(week,2) + solar_MW + wind_MW + temp + np.power(temp,2) + np.power(temp,3) + latitude + longitude']

linear_results = run_models(ols + elastic_nets, feature_ideas, df, folds=4, parallel=True)

HBox(children=(IntProgress(value=0, description='Feature Set', max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Models', max=10), HTML(value='')))

/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: 

/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: 

Exception in thread Thread-252:
Traceback (most recent call last):
  File "/home/hobbs/.conda/envs/ugh/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/hobbs/.local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/hobbs/.conda/envs/ugh/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very sm

HBox(children=(IntProgress(value=0, description='Models', max=10), HTML(value='')))

/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: 

HBox(children=(IntProgress(value=0, description='Models', max=10), HTML(value='')))

/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/hobbs/.conda/envs/ugh/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: 

Now we can calculate the weekly mean average error (WMAE).

In [132]:
def add_wmae(result_dict):
    wmae_list = [np.mean(wmae(error_list[0], df)) for error_list in result_dict['errors']]

    result_dict['results']['wmae'] = wmae_list
    
    return result_dict

linear_results = add_wmae(linear_results)

linear_results['results'].to_pickle('linear_results.p')

linear_results['results']

,features,model,score,wmae
0,dollar_mw ~ price_last_hr + price_yesterday + ...,"LinearRegression(copy_X=True, fit_intercept=Tr...",1.984488e+01,1.558057e-01
1,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.5, copy_X=True, fit_interce...",2.126031e+01,1.826296e-01
2,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.5, copy_X=True, fit_interce...",2.040441e+01,1.682655e-01
3,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.5, copy_X=True, fit_interce...",1.985613e+01,1.576113e-01
4,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.75, copy_X=True, fit_interc...",2.221879e+01,1.982394e-01
5,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.75, copy_X=True, fit_interc...",2.088021e+01,1.769070e-01
6,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=0.75, copy_X=True, fit_interc...",1.987007e+01,1.587910e-01
7,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=1.0, copy_X=True, fit_interce...",2.313768e+01,2.130118e-01
8,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=1.0, copy_X=True, fit_interce...",2.139897e+01,1.857998e-01
9,dollar_mw ~ price_last_hr + price_yesterday + ...,"ElasticNet(alpha=1.0, copy_X=True, fit_interce...",1.988953e+01,1.601693e-01


# Nonlinear Model

In [134]:
feature_ideas = ['dollar_mw ~ opr_hr + week + day + solar_MW + wind_MW + temp + latitude + longitude',
                 'dollar_mw ~ opr_hr + week + day + solar_MW + wind_MW + latitude + longitude + price_last_hr + price_yesterday + price_last_week']

rf = ensemble.RandomForestRegressor(n_jobs = 4)
gb = ensemble.GradientBoostingRegressor(max_depth = 10)
models = [rf, gb]

nonlinear_results = run_models(models, feature_ideas, df)

nonlinear_results = add_wmae(nonlinear_results)

nonlinear_results['results'].to_pickle('nonlinear_results.p')

nonlinear_results['results']

HBox(children=(IntProgress(value=0, description='Feature Set', max=2), HTML(value='')))

Exception in thread Thread-362:
Traceback (most recent call last):
  File "/home/hobbs/.conda/envs/ugh/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/hobbs/.local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/hobbs/.conda/envs/ugh/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



HBox(children=(IntProgress(value=0, description='Models', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Models', max=2), HTML(value='')))

,features,model,score,wmae
0,dollar_mw ~ opr_hr + week + day + solar_MW + w...,"RandomForestRegressor(bootstrap=True, criterio...",7.537054,0.054098
1,dollar_mw ~ opr_hr + week + day + solar_MW + w...,"GradientBoostingRegressor(alpha=0.9, criterion...",7.466121,0.063105
2,dollar_mw ~ opr_hr + week + day + solar_MW + w...,"RandomForestRegressor(bootstrap=True, criterio...",5.344259,0.025295
3,dollar_mw ~ opr_hr + week + day + solar_MW + w...,"GradientBoostingRegressor(alpha=0.9, criterion...",5.267206,0.038049


# Testing

Now we can test our chosen models on the held-out data.

In [136]:
ols = linear_model.LinearRegression(fit_intercept=True, n_jobs = 3)

best_nonlinear_features = 'dollar_mw ~ opr_hr + week + day + solar_MW + wind_MW + latitude + longitude + price_last_hr + price_yesterday + price_last_week'
best_linear_features = 'dollar_mw ~ price_last_hr + price_yesterday + price_last_week'

# Train on full set
def full_train_test(df, features, model):
    y, X = patsy.dmatrices(features, data=df, return_type = 'dataframe')
    X = preprocessing.scale(X)
    fitted = model.fit(X, np.ravel(y))
    y_test, X_test = patsy.dmatrices(features, data=df_test, return_type = 'dataframe')
    # Save indices from y
    index_values = y_test.index
    # Calculate y_hats and map to indices
    X_test = preprocessing.scale(X_test)
    y_hat = pd.DataFrame(data = fitted.predict(X_test), index = index_values)
    errors = np.subtract(y_test, y_hat)
    return {'model': model, 'rmse': rmse(y_test, y_hat), 'errors': errors}

final_linear = full_train_test(df, best_nonlinear_features, ols)
final_nonlinear = full_train_test(df, best_nonlinear_features, gb)
#final_linear = add_wmae(final_linear)

print("linear: ", np.mean(wmae(final_linear['errors'], df_test)))
print("nonlinear: ", np.mean(wmae(final_nonlinear['errors'], df_test)))

linear:  0.21884839356
nonlinear:  0.213111297858


# Graphics

This graphic charts the relationship between temperatures and prices.

In [ ]:
normalize = lambda x: (x - np.mean(x)) / np.std(x)

avg_price = pd.DataFrame(df[['dollar_mw', 'temp']].groupby(level = 'time').mean())
avg_price['time'] = avg_price.index.values

In [ ]:
avg_price[['temp', 'dollar_mw']] = avg_price[['temp', 'dollar_mw']].apply(normalize)

ggplot(avg_price, aes(x = 'time')) + \
    geom_line(aes(y = 'dollar_mw')) + \
    geom_line(aes(y = 'temp', color = 'red'))

In [ ]:
feature_ideas = ['dollar_mw ~ price_last_hr + price_yesterday + price_last_week']

rf = ensemble.RandomForestRegressor(n_jobs = 4)
gb = ensemble.GradientBoostingRegressor(max_depth = 10)
models = [rf, gb]

nonlinear_results = run_models(models, feature_ideas, df)
nonlinear_results.to_pickle('new_nonlinear_results.p')

In [ ]:
ols = [linear_model.LinearRegression(fit_intercept=False, n_jobs = 3)]

#generate a huge list of different elastic nets
elastic_nets = [linear_model.ElasticNet(alpha= a, l1_ratio = l, warm_start = True)
                for a, l in list(itertools.product(np.linspace(0.5, 1, num = 3), np.linspace(0,1, num = 3)))]

feature_ideas = ['dollar_mw ~ C(node)*(price_last_hr + price_yesterday + price_last_week)']

linear_results = run_models(ols, feature_ideas, df, folds=4, parallel=True)

linear_results.to_pickle('new_linear_results.p')

linear_results.head()

In [ ]:
# A single
linear_results.head()

In [ ]:
import statsmodels.formula.api as smf

model = 'dollar_mw ~ C(node)*(price_last_hr + price_yesterday + price_last_week)'

model = smf.ols(model, data = df)
res = model.fit()
res.summary()

In [ ]:
import statsmodels.formula.api as smf

model = 'dollar_mw ~ price_last_hr + price_yesterday + price_last_week'

model = smf.ols(model, data = df.loc['ALAMT4G_7_B1'])
res = model.fit()
res.summary()

In [ ]:
nonlinear_results.iloc[0]['errors'][0]

In [ ]:
nonlinear_results.iloc[0]['errors'][0]